### Load the metadata

In [1]:
import pandas as pd
import numpy as np
import os

metadata_mapping = pd.read_excel(os.path.join(os.getcwd(),'FAA DRS','DRS Document Types Metadata Mapping.xlsx')) # from https://drs.faa.gov/help/helpdetails and https://drs.faa.gov/help/helpdetails

doc_types = pd.unique(metadata_mapping.loc[:, "Document Type Name in API request"])
doc_types = [val for val in doc_types if isinstance(val, str)]

### STC
merge metadata, raw text and extracted text.

In [9]:
import glob
import os
import pandas as pd

df_stc_pdftext_googlevision = pd.read_parquet(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\database\decoded_pdf_using_text-from-pdf-googlevision.parquet")
#df_stc_pdftext_pdfplumber = pd.read_parquet(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\database\decoded_pdf_using_text-from-pdf-pdfplumber.parquet")

df_stc_metadata = pd.read_parquet(glob.glob(os.path.join(os.getcwd(), 'database', 'all drs', 'stc*.parquet'))[-1])
df_stc_metadata['documentGuid'] = df_stc_metadata['documentGuid'].astype('string')

df_stc_metadata_and_text = df_stc_metadata.join(df_stc_pdftext_googlevision.set_index('documentGuid'), on="documentGuid", how="right", lsuffix="_metadata", rsuffix="_googlevision")
df_stc_metadata_and_text.to_parquet(os.path.join(os.getcwd(),'database','stc_metadata_and_text.parquet'))

c:\Users\victor\anaconda3\envs\faa-nlp-drs\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


### Any other DRS document
merge metadata and raw text

In [ ]:
import os
import glob
import pandas as pd
import logging
import pdfplumber

logging.basicConfig(filename='18_merge_metadata_and_text.log', encoding='utf-8',format='%(asctime)s %(message)s', level=logging.INFO)

# for all doc types

for ocr_option in ['googlevision', 'pdfplumber']:
    text_source = 'text-from-pdf-'+ocr_option

    if ocr_option=='pdfplumber':
        available_txts = glob.glob(os.path.join(os.getcwd(),'database','data','stc','pdf','*.pdf'))
    elif ocr_option=='googlevision':
        available_txts = glob.glob(os.path.join(os.getcwd(),'database','data','stc','text-from-pdf','*.txt'))

    decoded_pdf = [[]]

    for path_file in available_txts:
        try:
            if ocr_option=='pdfplumber':
                with pdfplumber.open(path_file) as opened_pdf:    
                    raw_text_pagesplit = [str(this_page.extract_text())+'\n' for this_page in opened_pdf.pages] 
            elif ocr_option=='googlevision':
                with open(path_file, 'r', encoding="utf8") as f:
                    raw_text = f.read()
                    raw_text_pagesplit = raw_text.split("\n\n")

            decoded_pdf.append([path_file.split('__')[-1][:-4],
                                list2text_frompdf(get_all_descriptions(raw_text_pagesplit)),
                                list2text_frompdf(get_all_limitations(raw_text_pagesplit)),
                                "\n\n".join(raw_text_pagesplit)
                                ])

        except Exception as e:
            logging.info('Failed for '+path_file+' with Exception: '+str(e))
            decoded_pdf.append([path_file.split('__')[-1][:-4],
                                '',
                                '',
                                ''])

    df_pdf_content = pd.DataFrame(decoded_pdf,
                                columns=['documentGuid', 
                                        'descriptions', 
                                        'limitations',
                                        'rawtext'])    
    df_pdf_content = df_pdf_content.astype("string")

    df_pdf_content.to_parquet(os.path.join(os.getcwd(),'database','decoded_pdf_using_'+text_source+'.parquet'))


In [3]:
import glob

#df_stc_pdftext_googlevision = pd.read_excel(os.path.join(os.getcwd(),'database','decoded_pdf_using_text-from-googlevision.parquet'))
#df_stc_pdftext_pdfplumber = pd.read_excel(os.path.join(os.getcwd(),'database','decoded_pdf_using_text-from-pdfplumber.parquet'))

df_stc_metadata = pd.read_parquet(glob.glob(os.path.join(os.getcwd(), 'database', 'all drs', 'stc*.parquet'))[-1])
df_stc_metadata['documentGuid']=df_stc_metadata['documentGuid'].astype('str')

df_stc_metadata_and_text = df_stc_metadata.join(df_stc_pdftext_googlevision.set_index('documentGuid'), on="documentGuid", how="right", lsuffix="_metadata", rsuffix="_googlevision")
df_stc_metadata_and_text = df_stc_metadata.join(df_stc_pdftext_pdfplumber.set_index('documentGuid'), on="documentGuid", how="right", lsuffix="", rsuffix="_pdfplumber")

df_stc_metadata_and_text.to_parquet(os.path.join(os.getcwd(),'database','stc_metadata_and_text.parquet'))

NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:630:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:485:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.maximizeWindow@chrome://remote/content/marionette/driver.sys.mjs:2619:15
despatch@chrome://remote/content/marionette/server.sys.mjs:318:40
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20
